# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 27 2022 9:36AM,247355,15,8575944,"Brookfield Pharmaceuticals, LLC",Released
1,Sep 27 2022 9:36AM,247355,15,8575945,"Brookfield Pharmaceuticals, LLC",Released
2,Sep 27 2022 9:36AM,247355,15,8575946,"Brookfield Pharmaceuticals, LLC",Released
3,Sep 27 2022 9:36AM,247355,15,8575947,"Brookfield Pharmaceuticals, LLC",Released
4,Sep 27 2022 9:36AM,247355,15,8575948,"Brookfield Pharmaceuticals, LLC",Released
5,Sep 27 2022 9:36AM,247355,15,8575949,"Brookfield Pharmaceuticals, LLC",Released
6,Sep 27 2022 9:36AM,247355,15,8575950,"Brookfield Pharmaceuticals, LLC",Released
7,Sep 27 2022 9:36AM,247355,15,8575951,"Brookfield Pharmaceuticals, LLC",Released
8,Sep 27 2022 9:36AM,247355,15,8575952,"Brookfield Pharmaceuticals, LLC",Released
9,Sep 27 2022 9:36AM,247355,15,8575953,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,247348,Released,55
29,247352,Released,67
30,247353,Released,6
31,247354,Released,17
32,247355,Released,39


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247348,NaN,NaN,55.0
247352,NaN,NaN,67.0
247353,NaN,NaN,6.0
247354,NaN,NaN,17.0
247355,NaN,NaN,39.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247254,0.0,6.0,1.0
247258,0.0,0.0,1.0
247260,1.0,0.0,5.0
247262,18.0,15.0,18.0
247293,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247254,0,6,1
247258,0,0,1
247260,1,0,5
247262,18,15,18
247293,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247254,0,6,1
1,247258,0,0,1
2,247260,1,0,5
3,247262,18,15,18
4,247293,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247254,,6,1
1,247258,,,1
2,247260,1,,5
3,247262,18,15,18
4,247293,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 27 2022 9:36AM,247355,15,"Brookfield Pharmaceuticals, LLC"
39,Sep 27 2022 9:35AM,247348,10,ISDIN Corporation
94,Sep 27 2022 9:32AM,247354,12,Hush Hush
111,Sep 27 2022 9:24AM,247352,15,"Alliance Pharma, Inc."
178,Sep 27 2022 9:23AM,247353,20,"Exact-Rx, Inc."
184,Sep 27 2022 8:51AM,247345,10,"Senseonics, Incorporated"
185,Sep 27 2022 8:47AM,247344,10,ISDIN Corporation
199,Sep 27 2022 8:38AM,247340,20,"ACI Healthcare USA, Inc."
200,Sep 27 2022 8:36AM,247339,10,"Snap Medical Industries, LLC"
205,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 27 2022 9:36AM,247355,15,"Brookfield Pharmaceuticals, LLC",,,39
1,Sep 27 2022 9:35AM,247348,10,ISDIN Corporation,,,55
2,Sep 27 2022 9:32AM,247354,12,Hush Hush,,,17
3,Sep 27 2022 9:24AM,247352,15,"Alliance Pharma, Inc.",,,67
4,Sep 27 2022 9:23AM,247353,20,"Exact-Rx, Inc.",,,6
5,Sep 27 2022 8:51AM,247345,10,"Senseonics, Incorporated",,,1
6,Sep 27 2022 8:47AM,247344,10,ISDIN Corporation,,,14
7,Sep 27 2022 8:38AM,247340,20,"ACI Healthcare USA, Inc.",,,1
8,Sep 27 2022 8:36AM,247339,10,"Snap Medical Industries, LLC",,,5
9,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC",1,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 9:36AM,247355,15,"Brookfield Pharmaceuticals, LLC",39,,
1,Sep 27 2022 9:35AM,247348,10,ISDIN Corporation,55,,
2,Sep 27 2022 9:32AM,247354,12,Hush Hush,17,,
3,Sep 27 2022 9:24AM,247352,15,"Alliance Pharma, Inc.",67,,
4,Sep 27 2022 9:23AM,247353,20,"Exact-Rx, Inc.",6,,
5,Sep 27 2022 8:51AM,247345,10,"Senseonics, Incorporated",1,,
6,Sep 27 2022 8:47AM,247344,10,ISDIN Corporation,14,,
7,Sep 27 2022 8:38AM,247340,20,"ACI Healthcare USA, Inc.",1,,
8,Sep 27 2022 8:36AM,247339,10,"Snap Medical Industries, LLC",5,,
9,Sep 27 2022 8:30AM,247331,10,"Virtue Rx, LLC",2,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 9:36AM,247355,15,"Brookfield Pharmaceuticals, LLC",39,,
1,Sep 27 2022 9:35AM,247348,10,ISDIN Corporation,55,,
2,Sep 27 2022 9:32AM,247354,12,Hush Hush,17,,
3,Sep 27 2022 9:24AM,247352,15,"Alliance Pharma, Inc.",67,,
4,Sep 27 2022 9:23AM,247353,20,"Exact-Rx, Inc.",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 9:36AM,247355,15,"Brookfield Pharmaceuticals, LLC",39.0,NaN,NaN
1,Sep 27 2022 9:35AM,247348,10,ISDIN Corporation,55.0,NaN,NaN
2,Sep 27 2022 9:32AM,247354,12,Hush Hush,17.0,NaN,NaN
3,Sep 27 2022 9:24AM,247352,15,"Alliance Pharma, Inc.",67.0,NaN,NaN
4,Sep 27 2022 9:23AM,247353,20,"Exact-Rx, Inc.",6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 9:36AM,247355,15,"Brookfield Pharmaceuticals, LLC",39.0,0.0,0.0
1,Sep 27 2022 9:35AM,247348,10,ISDIN Corporation,55.0,0.0,0.0
2,Sep 27 2022 9:32AM,247354,12,Hush Hush,17.0,0.0,0.0
3,Sep 27 2022 9:24AM,247352,15,"Alliance Pharma, Inc.",67.0,0.0,0.0
4,Sep 27 2022 9:23AM,247353,20,"Exact-Rx, Inc.",6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2967791,99.0,20.0,2.0
12,247354,17.0,0.0,0.0
15,494707,106.0,0.0,0.0
19,989194,41.0,7.0,17.0
20,1236586,26.0,16.0,18.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2967791,99.0,20.0,2.0
1,12,247354,17.0,0.0,0.0
2,15,494707,106.0,0.0,0.0
3,19,989194,41.0,7.0,17.0
4,20,1236586,26.0,16.0,18.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,99.0,20.0,2.0
1,12,17.0,0.0,0.0
2,15,106.0,0.0,0.0
3,19,41.0,7.0,17.0
4,20,26.0,16.0,18.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,99.0
1,12,Released,17.0
2,15,Released,106.0
3,19,Released,41.0
4,20,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Completed,2.0,0.0,0.0,17.0,18.0
Executing,20.0,0.0,0.0,7.0,16.0
Released,99.0,17.0,106.0,41.0,26.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Completed,2.0,0.0,0.0,17.0,18.0
1,Executing,20.0,0.0,0.0,7.0,16.0
2,Released,99.0,17.0,106.0,41.0,26.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Completed,2.0,0.0,0.0,17.0,18.0
1,Executing,20.0,0.0,0.0,7.0,16.0
2,Released,99.0,17.0,106.0,41.0,26.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()